In [2]:
import os
import sys
import time
import cv2
import numpy as np
import tensorflow as tf
from direct_key import UP,Down,no_key
from key_check import key_check,keys_to_output
from grab_screen import grab_screen
from testing_model import testing_model
from keras.models import load_model, Model,Sequential,model_from_json
from keras.layers import Dense, Activation, Dropout, Input, LSTM, Reshape, Lambda, RepeatVector
from keras.initializers import glorot_uniform
from keras.utils import to_categorical
from keras.optimizers import Adam
from keras import backend as K
import numpy as np
import pandas as pd
from collections import Counter

In [3]:
detection_graph = tf.Graph()
with detection_graph.as_default():
    od_graph_def = tf.GraphDef()
    with tf.gfile.GFile("frozen_inference_graph.pb", 'rb') as fid:
        serialized_graph = fid.read()
        od_graph_def.ParseFromString(serialized_graph)
        tf.import_graph_def(od_graph_def, name='')

In [4]:
file_name = 'trainig_data_lstm.npy'
if os.path.isfile(file_name):
    print('File exists, loading previous data!')
    training_data = list(np.load(file_name,allow_pickle=True))
else:
    print('File does not exist, starting fresh!')
    training_data = []

File exists, loading previous data!


In [5]:
with detection_graph.as_default():
    with tf.Session(graph=detection_graph) as sess:
        image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')
        feature_vector = detection_graph.get_tensor_by_name("FeatureExtractor/MobilenetV1/Conv2d_13_pointwise_2_Conv2d_5_3x3_s2_128/Relu6:0")
        for i in list(range(3))[::-1]:
            print(i + 1)
            time.sleep(1)
        while True:
            screen = grab_screen(region=(0,40,800,640))
            image_np = cv2.resize(screen, (80, 60))
            cv2.imshow('',image_np)
            image_np_expanded = np.expand_dims(image_np, axis=0)
            (rep) = sess.run([feature_vector],feed_dict={image_tensor: image_np_expanded})
            keys = key_check()
            output = keys_to_output(keys)
            print(np.shape(rep))
            training_data.append([rep,output])
            if len(training_data) % 100 == 0:
                print(len(training_data))
                print(np.shape(training_data))
                np.save(file_name, training_data)
            if cv2.waitKey(25) & 0xFF == ord('q'):
                cv2.destroyAllWindows()
                break

3
2
1
output is : [0, 1, 0]
(1, 1, 1, 1, 128)
output is : [0, 1, 0]
(1, 1, 1, 1, 128)
output is : [0, 1, 0]
(1, 1, 1, 1, 128)


KeyboardInterrupt: 

In [ ]:
train_data = list(np.load('trainig_data_lstm.npy',allow_pickle=True))
df=pd.DataFrame(train_data)
print(df)
print(Counter(df[1].apply(str)))
up = []
down = []
no_action = []
for data in train_data:
    img = data[0]
    choice = data[1]
    if choice == [1,0,0]:
        up.append([img,choice])
    elif choice == [0,1,0]:
        no_action.append([img,choice])
    elif choice == [0,0,1]:
        down.append([img,choice])
    else:
        print('no matches')
no_action = no_action[:len(up)][:len(down)]
up = up[:len(no_action)]
down = down[:len(no_action)]
final_data = no_action + up + down
np.save('training_data_v.npy', train_data)

In [ ]:
steps_of_history = 5

In [ ]:
def get_model_movement():
    model = Sequential()  
    model.add(LSTM(60, input_shape=(5,128),return_sequences=True))
    model.add(Dropout(0.6))
    model.add(LSTM(60,return_sequences=False))
    model.add(Dropout(.6))
    model.add(Dense(3))
    model.compile(loss='mean_absolute_error', optimizer='adam',metrics=['accuracy'])
    return model

In [ ]:
def reshape_for_lstm(data):
    trainX = []
    trainY_movement = []
    trainY_action = []
    for i in range(0, len(data) - steps_of_history):
        window = data[i:i + steps_of_history]
        sampleX = []
        for row in window:
            sampleX.append(row[0])
        sampleY_movement = np.array(window[-1][1]).reshape(-1)
        trainX.append(np.array(sampleX).reshape(steps_of_history, -1))
        trainY_movement.append(sampleY_movement)
    print(np.array(trainX).shape)
    print(np.array(trainY_movement).shape)
    return trainX, list(trainY_movement)

In [ ]:
def train_lstm():
    filename = "training_data_v.npy"
    data = list(np.load(filename,allow_pickle=True))
    train = 1
    if train == 1:
        with detection_graph.as_default(): 
            trainX, trainY_movement= reshape_for_lstm(data)
            model = get_model_movement()
            model.fit(np.array(trainX),np.array(trainY_movement), nb_epoch=200, batch_size=100)
            model.save("model_lstm_1",model)

In [ ]:
train_lstm()

In [ ]:
tf.reset_default_graph()
with detection_graph.as_default():
    with tf.Session(graph=detection_graph) as sess:
        steps_of_history = 5
        input_window = np.zeros(shape=(steps_of_history, 128))
        model=get_model_movement()
        model=load_model("model_lstm_1")
        up_thresh=.80
        do_thresh=.80
        n_thresh=.80
        image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')
        feature_vector = detection_graph.get_tensor_by_name("FeatureExtractor/MobilenetV1/Conv2d_13_pointwise_2_Conv2d_5_3x3_s2_128/Relu6:0")
        for i in list(range(3))[::-1]:
            print(i + 1)
            time.sleep(1)
        while True:
            screen = grab_screen(region=(0,40,800,640))
            image_np = cv2.resize(screen, (80, 60))
            cv2.imshow('',image_np)
            image_np_expanded = np.expand_dims(image_np, axis=0)
            (rep) = sess.run([feature_vector],feed_dict={image_tensor: image_np_expanded})
            input_window[:-1, :] = input_window[1:, :]
            input_window[-1, :] = np.array(rep).reshape(-1, 128)
            prediction = model.predict(input_window.reshape(-1,5, 128))
            print(prediction)
            if prediction[0][0]>up_thresh:
                UP()
                print("up action took")
            elif prediction[0][2] > do_thresh:
                Down()
                print("down action took")
            elif prediction[0][1] > n_thresh:
                no_key()
                print("no action took")
            if cv2.waitKey(25) & 0xFF == ord('q'):
                cv2.destroyAllWindows()
                break     
